In [3]:
import numpy as np
import sys
import scipy
from scipy.optimize import curve_fit
from scipy.stats import unitary_group
import time as t
import matplotlib.pyplot as plt
np.set_printoptions(precision=2, linewidth=100)


In [ ]:
class DecisionTree():
    def __init__(self, data = None):
        self.nodes = []
        self.data = data
        
#define the decision tree of the game
class LoanGameTree():
    def __init__(self):
        root = DecisionTree(["lender","creation -> funding -> giving"])
        root.nodes.append(DecisionTree(["lender","lender cheat give"]))
        root.nodes[0].nodes.append(DecisionTree(["exit","blameCannotGive(0)"]))
        root.nodes[0].nodes.append(DecisionTree(["exit","blameRenterRecieved(0)"]))
        root.nodes[0].nodes.append(DecisionTree(["exit","abortCouldntGive(0)"]))
        root.nodes.append(DecisionTree(["exit","abortTimeoutGiving()"]))
        root.nodes.append(DecisionTree(["renter","lender truth give"]))
        root.nodes[2].nodes.append(DecisionTree(["lender","renter cheat get"]))
        root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","blameCannotGive(1)"]))
        root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","blameRenterRecieved(1)"]))
        root.nodes[2].nodes[0].nodes.append(DecisionTree(["exit","abortCouldntGive(1)"]))
        root.nodes[2].nodes.append(DecisionTree(["lender","renter truth get"]))
        root.nodes[2].nodes[1].nodes.append(DecisionTree(["exit","abortTimeoutForcePurchase()"]))
        root.nodes[2].nodes[1].nodes.append(DecisionTree(["renter","lender try to return"]))
        root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["lender","renter cheat"]))
        root.nodes[2].nodes[1].nodes[1].nodes[0].nodes.append(DecisionTree(["exit","blameFalseReturn(1)"]))
        root.nodes[2].nodes[1].nodes[1].nodes[0].nodes.append(DecisionTree(["exit","Finalize(1)"]))
        root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["exit","abortTimeoutForcePurchase()"]))
        root.nodes[2].nodes[1].nodes[1].nodes.append(DecisionTree(["lender","renter truth"]))
        root.nodes[2].nodes[1].nodes[1].nodes[2].nodes.append(DecisionTree(["exit","blameFalseReturn(0)"]))
        root.nodes[2].nodes[1].nodes[1].nodes[2].nodes.append(DecisionTree(["exit","Finalize(0)"]))
        
        
        

In [ ]:
players = 10

lenders = np.zeros((players,2))
renters = np.zeros((players,2))
